In [9]:
from google.cloud import storage
client = storage.Client()

bucket_name = "tdt4173-datasets"
bucket = client.get_bucket(bucket_name)

blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

cats-vs-dogs/
cats-vs-dogs/raw-data/
cats-vs-dogs/raw-data/catsanddogs.zip


In [17]:
blob_name = "cats-vs-dogs/raw-data/catsanddogs.zip"
blob = bucket.get_blob(blob_name)
blob.download_to_filename("data/catsanddogs.zip")

In [59]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [88]:
class DogsAndCats:
    IMG_SIZE = 128
    CATS = "data/PetImages/Cat"
    DOGS = "data/PetImages/Dog"
    TESTING = "data/PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []
    
    counts = {CATS: 0, DOGS: 0}
    
    def make_training_data(self):
        for label, classnum in self.LABELS.items():
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        h, w = img.shape
                        img = cv2.copyMakeBorder(
                            img,
                            top=0,
                            right=max(0, h-w),
                            bottom=max(0, w-h),
                            left=0,
                            borderType=0,
                        )
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                        
                        self.counts[label] += 1
                        
                    except Exception as e:
                        pass
                    
                    
        np.random.shuffle(self.training_data)
        np.save("data/processed/training_data.npy", self.training_data)
        print(f"Cats: {self.counts[self.CATS]}, dogs: {self.counts[self.DOGS]}")

DogsAndCats().make_training_data()

100%|██████████| 12501/12501 [00:17<00:00, 717.13it/s]


Cats: 12476, dogs: 12470


In [89]:
blob_name = "cats-vs-dogs/processed/catsvsdogs.npy"
blob = bucket.blob(blob_name)

source_file_name = "data/processed/training_data.npy"
blob.upload_from_filename(source_file_name)